Question 1


In [ ]:
library(knitr)

# Defining the sample space on the genotypes for both independent sites 
sample_space <- data.frame(
  site1 = rep(c("00", "01", "11"), each = 3),
  site2 = rep(c("00", "01", "11"), times = 3)
) #calculating the occurrences of 11's across the rows 
sample_space$G <- rowSums(matrix(unlist(strsplit(paste(sample_space$site1, sample_space$site2), " ")), ncol = 2) == "11")

# Manually define G counts and corresponding probabilities
pmf_df <- data.frame(
  G = c("0", "1", "2", "3", "4"),
  Probability = c(1/9, 2/9, 3/9, 2/9, 1/9)
)

# Print the PMF in the desired format using knitr's kable function
cat("Probability Mass Function (PMF) of G:\n")
knitr::kable(pmf_df, align = "c")




Question 1.2

In [ ]:
# Manually define G counts and corresponding probabilities
pmf_df <- data.frame(
  G = c("0", "1", "2", "3", "4"),
  Probability = c(1/9, 2/9, 3/9, 2/9, 1/9)
)

# Calculate Cumulative Distribution Function (CDF)
pmf_df$Cumulative_Prob <- cumsum(pmf_df$Probability)

# Create step function for CDF and plot 
plot(pmf_df$G, pmf_df$Cumulative_Prob, type = "s", col = "#000000", lwd = 5,
     xlab = "Alternative Allele counts", ylab = "Cumulative Probability", main = "Cumulative Distribution Function (CDF) of G")
plot(as.numeric(pmf_df$G), pmf_df$Probability,  type = "p", col = "#333333", pch = 19, xlab = "Alternative allele count", ylab = "Probabilities",
main = "Probability Mass Function (PMF) of G")






Question 2

In [ ]:
# Create the data frame with the given genotypes
data_2 <- data.frame(
  sample_2 = 1:10,
  myVariant1 = c("00", "00", "11", "01", "11", "00", "01", "00", "11", "11"),
  myVariant2 = c("01", "01", "00", "11", "01", "00", "01", "11", "00", "11")
)

# Convert genotypes to a numeric representation and calculate G
data_2$myG <- as.integer(substr(data_2$myVariant1, 1, 1)) + as.integer(substr(data_2$myVariant1, 2, 2)) +
              as.integer(substr(data_2$myVariant2, 1, 1)) + as.integer(substr(data_2$myVariant2, 2, 2))

# Calculate metrics: mean, standard deviation, and skew
new_sd <- sd(data_2$myG)
new_skewness <- sum((data_2$myG - mean(data_2$myG))^3) / (length(data_2$myG) * new_sd^3)
new_mean <- mean(data_2$myG)

# output the metrics of mean, skew and standard deviation 
cat("The Mean of the data is :", round(new_mean, digits = 6), "\n")
cat("The SD of the data is :", round(new_sd, digits = 6), "\n")
cat("The Skew of the data is:", round(new_skewness, digits = 6), "\n")

# generate a histogram
hist(data_2$myG, breaks = seq(min(data_2$myG) - 0.5, max(data_2$myG) + 0.5, by = 1), col = "#000000", border = "#8F9779",
     xlab = "Number of Alternate Alleles (G)", ylab = "Frequency", main = "Distribution of Variable G")


Question 2.2

In [ ]:
# outline the null and alternative hypothesis's
H0 <- "The mean of G is equal to the mean of G2"
H1 <- "The mean of G is not equal to the mean of G2"

# Perform a two-sample t-test
G2 <- c(2, 4, 2, 3, 3, 3, 2, 3, 3, 4)
t_test_result <- t.test(data_2$myG, G2)

# Report statement for the test
print("Task 2.2 Report:")
print(paste("Null hypothesis (H0):", H0))
print(paste("Alternative hypothesis (H1):", H1))
print("Test Result:")
rejected <- ifelse(t_test_result$p.value < 0.05, "is rejected", "cannot be rejected")
print(paste("The null hypothesis of means being equal", rejected, "at the significance level of 5%."))
t_stat <- round(t_test_result$statistic, digits = 6)
p_val <- round(t_test_result$p.value, digits = 6)
print(paste("The t-test for the means of Variable G and G2 gives a t-statistic of", t_stat,
            "with a p-value of", p_val))


Question 3

In [ ]:
install.packages("ggplot2")
library(ggplot2)


data_3 <- read.csv("assignment.csv")
data_3$income <- as.integer(data_3$income)
data_3$genotypes <- as.numeric(as.character(data_3$genotypes))
data_3$ancestry <- as.factor(data_3$ancestry)

# Fit first and second models
first_model <- lm(risk ~ genotypes + ancestry + income, data = data_3)
secound_model <- lm(risk ~ genotypes * ancestry, data = data_3)

# Create a scatter plot of genotypes vs. risk with different colors for each ancestry level
ggplot(data_3, aes(x = genotypes, y = risk, color = ancestry)) +
  geom_point() +
  labs(title = "Genotypes vs. Risk by Ancestry", x = "Genotypes", y = "Risk") +
  scale_color_manual(values = c("#FF5733", "#33FFC7", "#C733FF")) + 
  geom_smooth(data = subset(data_3, ancestry %in% c(1, 2, 3)), method = "lm", formula = y ~ x, aes(color = as.factor(ancestry), linetype = as.factor(ancestry)), se = FALSE) +  # Ancestry 1, 2, 3 regression lines with different line types
  scale_linetype_manual(values = c("dotted", "dashed", "dotdash")) +  # Different line types
  annotate("text", x = max(data_3$genotypes), y = max(data_3$risk), label = paste("First Model: ", round(summary(first_model)$adj.r.squared, 3)), hjust = 1) +  # Annotation for first model R-squared
  annotate("text", x = max(data_3$genotypes), y = min(data_3$risk), label = paste("Secound Model: ", round(summary(secound_model)$adj.r.squared, 3)), hjust = 1) +  # Annotation for second model R-squared
  theme_minimal()

summary(secound_model)
summary(first_model)


cat("Since income had no statistical significance (p = 0.86001), it was omitted from the model and it was taken out of the final model. 

Strong correlations between risk and genotypes, as well as ancestry groups 2 and 3, were revealed by the linear regression analysis. With a rise of around 4.9807 units per unit shift, genotypes were shown to have a considerable impact on risk; ancestry groups 2 and 3 contributed 4.3473 and 6.7463 units, respectively, over the baseline group (ancestry 1).

Highly significant p-values (*** for p < 0.001, ** for p < 0.01, * for p < 0.05) for genotypes, ancestry2, and ancestry3 indicated statistical significance. R-squared: 0.4244, adjusted R-squared: 0.4156) showed that the model fit well and explained around 42.44% of the variability in risk. The significance of predictors in explaining risk variation as a whole was highlighted by the F-statistic (48.17, p-value: < 2.2e-16).

Overall, this analysis—which was backed by strong statistical evidence and a well-fitting model—emphasized the critical roles that genotypes and particular ancestry groups play in predicting risk.")


Question 4

In [ ]:
# Read the CSV file
data <- read.csv("assignment.csv")

likelihood_mixture <- function(p, d) {
  pdf <- p[1] * dnorm(d, p[2], p[2] / 10) + (1 - p[1]) * dnorm(d, p[3], p[3] / 10)
  
  # Error handling to to avoid NAN error as multiplication of probabilities leads to 
    #numbers near zero so it is set at a predefined threshold 
  epsilon <- 1e-300
  pdf_safe <- ifelse(pdf < epsilon, epsilon, pdf)
  
  sum(log(pdf_safe))
}

estimate_parameters <- function(sd) {
  init_alpha <- 0.5
  init_mu1 <- 25
  init_mu2 <- 35
  # optimization is used to find the MLE of the observed likelihood observed in the data 
  optim(c(init_alpha, init_mu1, init_mu2), likelihood_mixture, d = sd, control = list(fnscale = -1))$par
}

bootstrap <- function(d, n, f) {
  replicate(n, f(sample(d, replace = TRUE)))
}

set.seed(100)  # seed value is set for reproducibility

#perform bootstrap resampling estimate the parameters
boot_results <- bootstrap(data$income, 1000, estimate_parameters)
point_estimates <- apply(boot_results, 1, mean)
standard_deviations <- apply(boot_results, 1, sd)

#computes the confidence intervals using quantiles
q <- function(r) quantile(r, c(0.025, 0.975))
alpha_ci <- q(boot_results[1, ])
mu1_ci <- q(boot_results[2, ])
mu2_ci <- q(boot_results[3, ])

cat("Point Estimates:", point_estimates, "\nStandard Deviations:", standard_deviations, "\nAlpha CI:", alpha_ci, "\nMu1 CI:", mu1_ci, "\nMu2 CI:", mu2_ci, "\n")

# Plot histogram with the title 
hist(data$income, freq = FALSE, col = "#000000", xlab = "Income", main = "Histogram of Income with Mixed Normal Distributions")

# puts the density curves for the estimated mixed model distributions 
curve(point_estimates[1] * dnorm(x, point_estimates[2], point_estimates[2] / 10) + 
      (1 - point_estimates[1]) * dnorm(x, point_estimates[3], point_estimates[3] / 10),
      col = "#FF0000", lwd = 2, add = TRUE)

# adds a legend to the plot 
legend("topright", legend = c("Mixed Normal Distribution 1", "Mixed Normal Distribution 2"), 
       col = c("#FF0000", "#000000"), lty = 1, lwd = 2)

